In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [2]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


video_path =  configParser.get('test_imagen','video_path')

ttwbdf =  int(configParser.get('test_imagen', 'time_to_wait_before_deleting_files'))


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [3]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [4]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [5]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [6]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('test_imagen', 'audio_length_used') 
model_filename =  configParser.get('test_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('test_imagen', 'openl3_mode')
folder =  configParser.get('test_imagen', 'folder')
number_of_images =  configParser.get('test_imagen', 'number_of_images')

In [7]:
age =  int(configParser.get('test_imagen', 'age'))
ethnicity =  str(configParser.get('test_imagen', 'ethnicity'))
gender =  str(configParser.get('test_imagen', 'gender'))
language =  str(configParser.get('test_imagen', 'language'))
image_guide_path =  str(configParser.get('test_imagen', 'image_guide_path'))
low_res_image =  str(configParser.get('test_imagen', 'low_res_image'))
blur_or_pixelate =  int(configParser.get('test_imagen', 'blur_or_pixelate'))

boxBlurMin =  int(configParser.get('test_imagen', 'boxBlurMin'))
boxBlurMax =  int(configParser.get('test_imagen', 'boxBlurMax'))

gaussianBlurMin =  int(configParser.get('test_imagen', 'gaussianBlurMin'))
gaussianBlurMax =  int(configParser.get('test_imagen', 'gaussianBlurMax'))

pix_to_min =  int(configParser.get('test_imagen', 'pix_to_min'))
pix_to_max =  int(configParser.get('test_imagen', 'pix_to_max'))

In [8]:
video_path

'/media/gamal/Passport/Datasets/VoxCeleb2TestMin/Voxceleb2TestVideo/id00017/01dfn2spqyE/00001.mp4'

In [9]:
dataGotten = [['1','1',video_path,age,ethnicity,gender,[],[],[],language,[]]]
df = pd.DataFrame(dataGotten,columns = ['ID','FACE_ID','VIDEO_PATH','caption_a','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','AUDIO_FEATURES','caption_l','image_path'])
df["image_guide"] = np.nan
df['low_res_image'] = low_res_image

In [10]:
df

,ID,FACE_ID,VIDEO_PATH,caption_a,caption_e,caption_g,SPEAKER_EMB,AUDIO_EMB,AUDIO_FEATURES,caption_l,image_path,image_guide,low_res_image
0,1,1,/media/gamal/Passport/Datasets/VoxCeleb2TestMi...,25,black,man,[],[],[],English,[],NaN,/media/gamal/Passport/Datasets/VoxCeleb2TestMi...


In [11]:
#import tensorflow as tf
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
#  tf.config.experimental.set_memory_growth(gpu, True)

In [12]:
#gpus
import pickle
import multiprocessing
multiprocessing.set_start_method('spawn')

In [13]:
from testing_imagen_speechbrain import extract_speechbrain_embeddings
from testing_imagen_pyannote_titanet import extract_pyannote_titanet_embeddings
from testing_imagen_audio_features import extract_audio_features
from testing_imagen_vision_transformer import extract_vision_transformer

audio_embs =  configParser.get('COMMON', 'audio_embs') 
from pydub import AudioSegment
import math
# Get original duration of video
audio = AudioSegment.from_file(video_path)
audio_length_og = math.floor(audio.duration_seconds)


video_filename = os.path.basename(video_path)

absPathAudio_w = os.path.abspath(audio_folder) + "/" + video_filename
absPathAudio = os.path.splitext(absPathAudio_w)[0]+'_audio.wav'
absPathAudio_w = os.path.splitext(absPathAudio_w)[0]


import subprocess
# Extract audio monochannel and with 16khz and put it in absPathAudio

command = "ffmpeg -nostats -loglevel 0 -y -i '" + video_path + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio  + "'"
subprocess.call(command, shell=True)

from threading import Thread
import time

# Function to delete audio temp files
def delFiles(filesToDelete):
    time.sleep(ttwbdf)  # wait a bit
    for file in filesToDelete:  
        try:
            os.remove(file)
        except OSError:
            pass


# Will either truncate or loop the original video to reach audio_length (6,12 or 24)
audio_length_list = [24]
for audio_length in audio_length_list:
    path_var_len_audio =  absPathAudio_w + "audio" + str(audio_length) + "s.wav"    # path to the variable length audio
    path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio

    if(audio_length_og > audio_length):
        # Truncate    

        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)


    else:
        # Loop then truncaate
        #print("lesa")
        twoDigitLenStr = f"{audio_length:02}"
        #print(twoDigitLenStr)
        command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
        subprocess.call(command, shell=True)
        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)




    q = multiprocessing.Queue()

    proc = multiprocessing.Process(target=extract_speechbrain_embeddings, args=(q,path_var_len_audio,))
    proc.start()
    proc.join()



    df['SPEAKER_EMB'] = q.get()
    proc = multiprocessing.Process(target=extract_pyannote_titanet_embeddings, args=(q,path_var_len_audio,audio_embs,audio_length,openl3_mode,output_folder,))
    proc.start()
    proc.join()

    with open(output_folder + '/' + 'audio_emb_pyannote_titanet.pickle', 'rb') as handle:
        df['AUDIO_EMB'] = pickle.load(handle)



    proc = multiprocessing.Process(target=extract_audio_features, args=(q,path_var_len_audio,output_folder,))
    proc.start()
    proc.join()



    with open(output_folder + '/' + 'audio_features.pickle', 'rb') as handle:
        df['AUDIO_FEATURES'] = pickle.load(handle)


    #proc = multiprocessing.Process(target=extract_vision_transformer, args=(q,output_folder,
    #                                                                        image_guide_path,output_folder,
    #    image_size,boxBlurMin,boxBlurMax,gaussianBlurMin,gaussianBlurMax,pix_to_min,pix_to_max,))
    #proc.start()
    #proc.join()



    #with open(output_folder + '/' + 'image_features.pickle', 'rb') as handle:
    #    df["image_guide"] = pickle.load(handle)
    #df["image_guide"] = np.zeros((1, 768))


    

    ftd = [absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp]
    tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
    tDelete.start()



2023-10-14 12:45:07.041382: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 12:45:07.084260: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
[NeMo W 2023-10-14 12:45:08 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-10-14 12:45:08 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at you

a
b
c


Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/17917d9c9cdff5c3c53e618a00299df26fd977aa/pytorch_model.bin`
Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/17917d9c9cdff5c3c53e618a00299df26fd977aa/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.
Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.
d


[NeMo W 2023-10-14 12:45:19 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-10-14 12:45:19 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-10-14 12:45:19 features:287] PADDING: 16
[NeMo I 2023-10-14 12:45:20 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/4e0b2d387a805da7c208b13d5898ee09de8ec1e9/speakerverification_en_titanet_large.nemo.
[NeMo I 2023-10-14 12:45:20 cloud:58] Found existing object /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
[NeMo I 2023-10-14 12:45:20 cloud:64] Re-using file from: /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo
[NeMo I 2023-10-14 12:45:20 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-10-14 12:45:20 features:287] PADDING: 16


[NeMo W 2023-10-14 12:45:20 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2023-10-14 12:45:20 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size:

[NeMo I 2023-10-14 12:45:20 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
e
f
g
h
i
j


In [14]:
df3 = df[["image_path","caption_a","caption_e","caption_g","caption_l"]]
data_frame = df3
data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
data_frame['AUDIO_EMB'] = df['AUDIO_EMB']
data_frame['AUDIO_FEATURES'] = df['AUDIO_FEATURES']
data_frame['image_guide'] = df['image_guide']
data_frame['low_res_image'] = df['low_res_image']

In [15]:
data_frame

,image_path,caption_a,caption_e,caption_g,caption_l,SPEAKER_EMB,AUDIO_EMB,AUDIO_FEATURES,image_guide,low_res_image
0,[],25,black,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...,NaN,/media/gamal/Passport/Datasets/VoxCeleb2TestMi...


In [16]:
import pickle


with open(output_folder + '/' + 'filename.pickle', 'wb') as handle:
    pickle.dump(data_frame, handle)

import os
os._exit(0)